<a id=“title_ID”></a>
# JWST Pipeline Validation Testing Notebook: Spec2, srctype, MIRI
Instruments Affected: MIRI

## Introduction and summary of test being run

This notebook processes an image through calwebb_spec2 and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file names.

2) Set the appropriate header keywords to test.

2) Run data through the source_type step in calwebb_spec2.

3) Compare input to output SRCTYPE keyword

## Documentation
The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

##Data used

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public?

Author: Mike Engesser

## Create Temporary Directory

In [ ]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

In [ ]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

## Set Up Import Statements

In [ ]:
from astropy.io import fits, ascii
import pytest
import numpy as np
import numpy.ma as ma

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_spec2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os
import sys

In [ ]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

## Print pipeline version number 

In [ ]:
jwst.__version__ 

## Read in data from Artifactory

In [ ]:
file = get_bigdata('jwst_validation_notebooks',
                      'validation_data',
                      'calwebb_spec2',
                      'spec2_miri_test',
                      'det_image_seq1_MIRIFUSHORT_12LONGexp1_rate.fits')

## Define functions for validation and running pipeline

In [ ]:
print('[*]', file=open('stpipe-log.cfg', "w"))
print('handler = file:pipeline.log', file=open('stpipe-log.cfg', "a"))
print('level = INFO', file=open('stpipe-log.cfg', "a"))

In [ ]:
def run_spec2(im):
    """Creates and runs an instance of the JWST spec2 pipeline. """
    
    #only assign_wcs and srctype are run
    spec2 = calwebb_spec2.Spec2Pipeline()
    spec2.bkg_subtract.skip = True
    spec2.assign_wcs.skip = False
    spec2.imprint_subtract.skip = True
    spec2.msa_flagging.skip = True
    spec2.extract_2d.skip = True
    spec2.wavecorr.skip = True
    spec2.srctype.skip = False
    spec2.straylight.skip = True
    spec2.fringe.skip = True
    spec2.pathloss.skip = True
    spec2.barshadow.skip = True
    spec2.photom.skip = True
    spec2.resample_spec.skip = True
    spec2.cube_build.skip = True
    spec2.extract_1d.skip = True
    spec2.flat_field.skip = True
    spec2.save_results = True
        
    spec2.run(im)

In [ ]:
def test_src_type(input_file, im1):
    """Passes data to spec2 pipeline and records the source type keyword of the output"""
    
    #run through spec2
    run_spec2(im1)
    
    #get the output file name
    cal_file = input_file.replace('rate.fits', 'cal.fits')
    
    #create an image model of the output and return the source type keyword
    im2 = ImageModel(cal_file)
    
    # Get the header source type value as well
    hdu=fits.open(cal_file)
    
    return im2.meta.target.source_type,hdu[1].header['SRCTYPE']

## Create an image model and test source type keywords

This test passes if none of the assertions return an error

In [ ]:
im = ImageModel(file)

In [ ]:
# Test the default case where no external information which should return EXTENDED
im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# Test the case where APT specified 'POINT'
im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

im.meta.target.source_type_apt = 'POINT'
    
src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
# Test the case where APT specified 'EXTENDED'
im.meta.target.source_type_apt = 'EXTENDED'
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = None

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If the background target keyword is set, it should always returns EXTENDED
# regardless of what APT said

im.meta.target.source_type_apt = 'POINT'
im.meta.observation.bkgdtarg = True

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If no information from APT, but dither keywords suggest a point source it should
# return POINT

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'POINT-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
# If no information from APT, but dither keywords suggest an extended source it should
# return EXTENDED

im.meta.target.source_type_apt = None
im.meta.observation.bkgdtarg = False
im.meta.dither.optimized_for = 'EXTENDED-SOURCE'

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'EXTENDED')
assert(src_type_hdr == 'EXTENDED')

In [ ]:
# If the observation is a TSO, return POINT

im.meta.observation.bkgdtarg = False
im.meta.visit.tsovisit = True
im.meta.exposure.nints = 3

src_type, src_type_hdr = test_src_type(file, im)

assert(src_type == 'POINT')
assert(src_type_hdr == 'POINT')

In [ ]:
print('No problems found.')